# Alzheimer Detection

In [2]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import itertools
import torch
%pip install efficientnet_pytorch
%pip install timm
from efficientnet_pytorch import EfficientNet

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Preprocess Data

In [4]:
# Define the directory where the data is stored
# Replace with the path to your dataset
data_dir = '/train/Mild_Impairment'

# Define transforms for the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.RandomGrayscale(p=0.1),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)

#with wandb.init(project='wound-detection'):
# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% of dataset
val_size = len(dataset) - train_size  # 20% of dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '/train/Mild_Impairment'

## Mean model: CNN

### Defining CNN architecture

In [5]:
# # Load a pre-trained model (e.g., ResNet18) and modify it for our task
model = models.resnet50(pretrained=True)

# Freeze all the layers in the pretrained model
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer with a new one that matches our number of classes (4 classes in our case)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4) 

# Move the model to the GPU if available
model = model.to(device)

C:\Users\asus\Downloads\Alzheimer_Detection-main\Alzheimer_Detection-main\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\asus\Downloads\Alzheimer_Detection-main\Alzheimer_Detection-main\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\asus/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:08<00:00, 12.4MB/s]


### Specify loss function and optimizer

In [6]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer, only optimizing the parameters of the final layer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

### Train the model

In [7]:
# Number of epochs to train for
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for inputs, labels in trainloader:
        # Move the input and label tensors to the correct device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print statistics
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")
    
    # Validation loss
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Loss: {val_loss / len(valloader)}, Accuracy: {100 * correct / total}%')
torch.save(model.state_dict(), '/kaggle/working/alzheimer_cnn_model.pth')
    

NameError: name 'trainloader' is not defined

## EfficientNet for improved performance

In [8]:
efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=4)

# Move the model to the GPU if available
efficientnet_model = efficientnet_model.to(device)

Loaded pretrained weights for efficientnet-b0


### Specify optimizer

In [9]:
optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)

### Train model

In [10]:
# Number of epochs to train for
num_epochs = 10

# train efficientnet model
for epoch in range(num_epochs):
    efficientnet_model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for inputs, labels in trainloader:
        # Move the input and label tensors to the correct device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = efficientnet_model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print statistics
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")
    
    # Validation loss
    efficientnet_model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = efficientnet_model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Loss: {val_loss / len(valloader)}, Accuracy: {100 * correct / total}%')

# Save the model
    
torch.save(efficientnet_model.state_dict(), '/kaggle/working/alzheimer_efficientnet_model.pth')


NameError: name 'trainloader' is not defined

## Vision Transformer for improved performance

In [22]:
# Load a pre-trained Vision Transformer model
vit_model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

# Move the model to the GPU if available
vit_model = vit_model.to(device)



Downloading: "https://github.com/facebookresearch/deit/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:63: UserWarning: Overwriting deit_tiny_patch16_224 in registry with models.deit_tiny_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_tiny_patch16_224(pretrained=False, **kwargs):
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:78: UserWarning: Overwriting deit_small_patch16_224 in registry with models.deit_small_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_small_patch16_224(pretrained=False, **kwargs):
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:93: UserWarning: Overwriting deit_base_patch16_224 in registry with models.deit_base_patch16_224. This is because the name being registered conflicts with an existing name. Plea

### Define loss function and optimizer

In [23]:
# Define the optimizer, only optimizing the parameters of the final layer
optimizer = optim.Adam(vit_model.parameters(), lr=0.001)

### Train Model

In [24]:
# Number of epochs to train for
num_epochs = 10

# train Vision Transformer model
for epoch in range(num_epochs):
    vit_model.train()  # Set the model to training mode
    running_loss = 0.0
    
    # torch.uint8 is supported on the CPU only, so we need to move the input and label tensors to the correct device
    for inputs, labels in trainloader:
        # Move the input and label tensors to the correct device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = vit_model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print statistics
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")
    
    # Validation loss
    vit_model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = vit_model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Loss: {val_loss / len(valloader)}, Accuracy: {100 * correct / total}%')
# Save the model
torch.save(vit_model.state_dict(), '/kaggle/working/alzheimer_vit_model.pth')

Epoch 1, Loss: 1.1634845238178968
Validation Loss: 1.0732713043689728, Accuracy: 32.1875%
Epoch 2, Loss: 1.0529922910034657
Validation Loss: 1.0259643480181695, Accuracy: 53.359375%
Epoch 3, Loss: 1.0125707723200321
Validation Loss: 0.8805326148867607, Accuracy: 56.25%
Epoch 4, Loss: 1.0047540474683045
Validation Loss: 0.9534257456660271, Accuracy: 53.59375%
Epoch 5, Loss: 0.9667909078299999
Validation Loss: 0.8876874297857285, Accuracy: 55.703125%
Epoch 6, Loss: 0.9608717679977417
Validation Loss: 0.9172666013240814, Accuracy: 58.515625%
Epoch 7, Loss: 0.9600563608109951
Validation Loss: 0.8975788906216622, Accuracy: 55.78125%
Epoch 8, Loss: 0.9425622124224902
Validation Loss: 0.9373615071177482, Accuracy: 54.921875%
Epoch 9, Loss: 0.9452154040336609
Validation Loss: 0.9440957054495811, Accuracy: 54.296875%
Epoch 10, Loss: 0.9265390537679196
Validation Loss: 0.8854614362120629, Accuracy: 56.953125%


## Model Evaluation

In [3]:
def plot_loss(train_losses, val_losses, title='Loss Plot'):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [4]:
def evaluate_model(model, dataloader, device, criterion):
    model.eval()  # Set model to evaluate mode
    val_losses = []
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_losses.append(loss.item())
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    # Calculate overall metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    # Plot loss and confusion matrix
    plot_loss([], val_losses, title='Validation Loss')  # Pass training losses if available
    plot_confusion_matrix(cm, classes=['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented'], title='Confusion Matrix')
    
    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')


In [ ]:
# Evaluating ResNet
evaluate_model(model, valloader, device, criterion)

# Evaluating  EfficientNet Model
evaluate_model(efficientnet_model, valloader, device, criterion)

# Evaluating Vision Transformer
evaluate_model(vit_model, valloader, device, criterion)